In [4]:
%load_ext autoreload
%autoreload 2

from src.jazz_ai import JazzAI
from sklearn.naive_bayes import MultinomialNB
import src.load_data as load_data
from IPython.display import display
import pandas as pd

# load in the data
training_data = load_data.data_pipeline()
mdl = JazzAI(MultinomialNB)
mdl.fit(training_data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,actual,pred,#4,1,2,3,4,5,6,7,b2,b3,b6,b7
4,4,3,0.012006,0.227929,0.136088,0.300244,0.092576,0.129756,0.016526,0.035858,0.013947,0.021195,0.005193,0.008682
5,1,5,0.072987,0.104716,0.131978,0.147286,0.059955,0.269796,0.035796,0.022771,0.018621,0.106464,0.020762,0.008869
6,1,2,0.007662,0.138722,0.229214,0.062018,0.037948,0.149190,0.105565,0.081696,0.025496,0.089811,0.022551,0.050127
7,5,4,0.099751,0.103346,0.072393,0.045390,0.192264,0.051092,0.004391,0.018950,0.172652,0.182663,0.032778,0.024330
8,2,b3,0.092394,0.073197,0.033084,0.015386,0.158314,0.043247,0.032759,0.014188,0.026008,0.292458,0.107986,0.110978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200803,7,2,0.005315,0.100495,0.296306,0.110390,0.013523,0.093131,0.084714,0.246746,0.012539,0.008607,0.003848,0.024386
200804,5,1,0.004896,0.376507,0.151907,0.052795,0.004508,0.139011,0.176950,0.038664,0.006626,0.008454,0.009123,0.030559
200805,4,6,0.030777,0.086199,0.111395,0.143160,0.038869,0.096549,0.266978,0.154819,0.003347,0.011169,0.037866,0.018873
200806,5,3,0.044365,0.045239,0.086732,0.409994,0.018913,0.275677,0.048909,0.023111,0.003472,0.030934,0.009116,0.003539


In [62]:
import numpy as np

from src.feature_matrix import create_feature_matrix

solo = training_data.query('melid==1')[['onset', 'key', 'chord', 'chord_root', 'melody_note', 'chord_3rd', 'chord_5th', 'chord_7th', 'chord_root_num']]
struct_df = solo[['chord_3rd', 'chord_5th', 'chord_7th', 'chord_root_num']].iloc[4:,]
init_notes = ["5", "4", "1", "b3"]

generated_notes = mdl.generate_solo(init_notes, struct_df)
# generated_notes.index = struct_df.index
generated_notes

,chord_3rd,chord_5th,chord_7th,chord_root_num,generated_notes
0,NaN,NaN,NaN,NaN,5
1,NaN,NaN,NaN,NaN,4
2,NaN,NaN,NaN,NaN,1
3,NaN,NaN,NaN,NaN,b3
4,1.0,1.0,1.0,11.0,5
...,...,...,...,...,...
525,1.0,1.0,1.0,11.0,b2
526,1.0,1.0,1.0,11.0,5
527,1.0,1.0,1.0,11.0,6
528,1.0,1.0,1.0,11.0,3


In [65]:
from src.music_theory import calculate_note_from_interval
import pandas as pd

inspect = pd.merge(solo, generated_notes, left_index=True, right_index=True, how='outer')[['melody_note', 'key', 'chord', 'chord_root', 'generated_notes']]
inspect = calculate_note_from_interval(inspect, 'chord_root', 'generated_notes', 'generated_note_names')
inspect

,melody_note,key,chord,chord_root,generated_notes,generated_note_names
0,F,Bb-maj,Bb6,Bb,5,F
1,D#,Bb-maj,Bb6,Bb,4,D#
2,A#,Bb-maj,Bb6,Bb,1,A#
3,C#,Bb-maj,Bb6,Bb,b3,C#
4,D#,Bb-maj,Bb6,Bb,5,F
...,...,...,...,...,...,...
525,F#,Bb-maj,Bb6,Bb,b2,B
526,F,Bb-maj,Bb6,Bb,5,F
527,D#,Bb-maj,Bb6,Bb,6,G
528,D,Bb-maj,Bb6,Bb,3,D
